In [ ]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
import pickle
import anvil.server

In [ ]:
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from feeds import BackFeed,DataFeed

In [ ]:
import fuzzy
from fuzzyrules import rule1_ed,rule2_ed,exit1_long_ed,exit1_short_ed

In [ ]:
def do_nothing(dfD):
    empty={t:0 for t in dfD}
    return empty,empty,empty
def always_buy(dfD):
    buy={t:1 for t in dfD}
    empty={t:0 for t in dfD}
    return buy,empty,empty
def always_sell(dfD):
    sell={t:-1 for t in dfD}
    empty={t:0 for t in dfD}
    return sell,empty,empty

In [1]:
class FuzzyStrat():
    def __init__(self,factor=1):
        self.fuz=fuzzy.Fuzzy()
        self.rules=[]
        self.exit_long_rules=[]
        self.exit_short_rules=[]
        self.model_type='fuzzy_rule_based'
        self.data_cols=['Close_n','datetime']
        self.factor=factor
        self.logL=[]
        self.add_rule('Rule1',rule1_ed,kwargs={'ifscale':.25,'thscale':1.0})
        self.add_rule('Exit1Long',exit1_long_ed,kwargs={'ifscale':.25,'thscale':1.0})
        self.add_rule('Exit1Short',exit1_short_ed,kwargs={'ifscale':.25,'thscale':1.0})
    def add_rule(self,name,rulef,kwargs={},weight=1,buy=1.13,sell=.89):
        self.rules+=[{'name':name,'func':rulef,'kwargs':kwargs,
                      'weight':weight,'buy':buy,'sell':sell}]
    def add_exit_long_rule(self,name,rulef,kwargs={},weight=1,buy=1,sell=.816):
        self.exit_long_rules+=[{'name':name,'func':rulef,'kwargs':kwargs,
                      'weight':weight,'buy':buy,'sell':sell}]
    def add_exit_short_rule(self,name,rulef,kwargs={},weight=1,buy=1.223,sell=1):
        self.exit_short_rules+=[{'name':name,'func':rulef,'kwargs':kwargs,
                      'weight':weight,'buy':buy,'sell':sell}]
    def check_entry_batch(self,dfD):
        decisionsD={t:0 for t in dfD}
        stopD={t:0 for t in dfD}
        targetD={t:0 for t in dfD}
        log_entry={}
        for t in dfD.keys():
            data=dfD[t]
            if data.shape[0]>2:
                r=data['Close_n'].values
                if self.factor>1: r=features.compress_prices(r,self.factor,ret='prices')
                ed,buy,sell=0,0,0
                for rule in self.rules:
                     ed+=rule['weight']*np.exp(rule['func'](r,self.fuz,**rule['kwargs']))
                     buy+=rule['buy']*rule['weight']
                     sell+=rule['sell']*rule['weight']
                if ed>buy or ed<sell: 
                    self.logL+=[(ed,dfD)]
                    # print(ed,buy,sell,np.sign(ed-1))
                    decisionsD[t]=np.sign(ed-1)
        return decisionsD,stopD,targetD#,ed
    def check_exit_batch(self,dfD,posf):
        def exit_fn(row):
            return self.exit_predicate(row,dfD[row.ticker])
        posf['to_exit']=posf.apply(exit_fn,axis=1).values
        return posf
    def exit_predicate(self,row,df):
        r=df['Close_n'].values
        if self.factor>1: r=features.compress_prices(r,self.factor,ret='prices')
        ed,exbuy,exsell=0,0,0
        if row['quant']>0:
            for rule in self.exit_long_rules:
                rv=np.exp(rule['func'](r,self.fuz,**rule['kwargs']))
                ed+=rule['weight']*rv
                exsell+=rule['sell']*rule['weight']
            # print(row['quant'],rv,features.mar(r,m=2,n=5),ed,exbuy,exsell)
            if ed<exsell:
                # print('exiting long',row['quant'],ed,exbuy,exsell)
                return True
        elif row['quant']<0:
            for rule in self.exit_short_rules:
                rv=np.exp(rule['func'](r,self.fuz,**rule['kwargs']))
                ed+=rule['weight']*rv
                exbuy+=rule['buy']*rule['weight']
            # print(row['quant'],rv,features.mar(r,m=2,n=5),ed,exbuy,exsell)
            if ed<exsell:
                # print('exiting long',row['quant'],ed,exbuy,exsell)
                return True
        else: return False
    def Check(strat,dfD):
        return strat.check_entry_batch(dfD)
    def Exit(strat,dfD,posf):
        return strat.check_exit_batch(dfD,posf)

In [ ]:
@anvil.server.callable
def check_entry_batch_f0_at_a_dot_edu(dfD):
    global fuzStrat
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return FuzzyStrat.Check(fuzStrat,dfDloc)

In [ ]:
@anvil.server.callable
def check_exit_batch_f0_at_a_dot_edu(dfD,posf):
    global fuzStrat
    posloc=pd.DataFrame(posf)
    dfDloc={}
    for t in dfD.keys():
        dfDloc[t]=pd.DataFrame(dfD[t])
    return FuzzyStrat.Exit(fuzStrat,dfDloc,posloc)